# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
3C OK
PASS OK
4C ? NN-value: PASS 0.853
4H OK
PASS OK
PASS OK
PASS ? NN-value: 5C 0.343
analyzing the play
CA


Loaded lib dds.dll


C8 OK
C3 OK
C5 OK
S8 OK
ST OK
SQ OK
SA OK
DK OK
DT OK
D6 OK
D3 OK
D9 OK
D7 OK
DA OK
D8 OK
DJ ?? losing: 0.70
DQ OK
HJ OK
S7 OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '3C',
 'candidates': [{'call': '3C', 'insta_score': 0.895}],
 'samples': ['8x.8x.Tx.AKJ9xxx A9.A9xx.QJxxx.Q8 QTxxx.KJTx.A9.Tx KJxx.Qxx.K8xx.xx 1.00000',
  '8x.8x.Tx.AKJ9xxx KQ9xx.xx.Q9xxx.Q AJTx.KT9x.KJ8.Tx xx.AQJxx.Axx.8xx 1.00000',
  '8x.8x.Tx.AKJ9xxx 9xx.AK9x.J8x.QTx AQxx.JT.AQxxx.8x KJTx.Qxxxx.K9x.x 1.00000',
  '8x.8x.Tx.AKJ9xxx Axxx.QJx.AJx.QT8 KT9x.A9xx.Q8xx.x QJx.KTxx.K9xx.xx 1.00000',
  '8x.8x.Tx.AKJ9xxx 9xx.AJTx.Axxx.xx ATxx.Qxx.K98xx.x KQJx.K9xx.QJ.QT8 1.00000',
  '8x.8x.Tx.AKJ9xxx AJ9x.AJx.Qxxx.8x Txx.KQ9xx.AJ9.Tx KQxx.Txx.K8xx.Qx 1.00000',
  '8x.8x.Tx.AKJ9xxx QJT.KQx.K98x.Txx 9xxx.AJTx.AJxx.Q AKxx.9xxx.Qxx.8x 1.00000',
  '8x.8x.Tx.AKJ9xxx 9x.QTxx.AQJ8xx.Q KQxxx.KJx.Kxx.8x AJTx.A9xx.9x.Txx 1.00000',
  '8x.8x.Tx.AKJ9xxx J9.ATxx.AJ9xx.xx Kxxx.KJ9x.xx.QT8 AQTxx.Qxx.KQ8x.x 1.00000',
  '8x.8x.Tx.AKJ9xxx 9x.KQxxx.AJx.T8x ATxx.T.K98xxx.Qx KQJxx.AJ9xx.Qx.x 1.00000',
  '8x.8x.Tx.AKJ9xxx AKJTx.Jxx.A8x.8x 9xx.AK9x.Q9xxx.T Qxx.QTxx.KJx.Qxx 1.00000',
  '8x.8x.Tx.AKJ9xxx KQJx.JT9x

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

{'card': 'CA',
 'candidates': [{'card': 'DT',
   'insta_score': 0.128,
   'expected_tricks_sd': 7.7,
   'p_make_contract': 0.1},
  {'card': 'CA',
   'insta_score': 0.863,
   'expected_tricks_sd': 7.77,
   'p_make_contract': 0.06}],
 'samples': ['KQJT9.T9xx.A98x. Axxxx..xxxx.QTxx x.AKQJxxx.KQJ.8x 0.12531',
  'AJ9xx.KT9.J9xx.Q KQxxx..Q8xx.T8xx T.AQJxxxxx.AKx.x 0.10501',
  'ATxxx.T9xx.KJ8x. KQ9xx..xxx.T8xxx J.AKQJxxx.AQ9x.Q 0.08908',
  'AK9x.9xx.QJ8xx.Q QJxxx..K9xx.Txxx Tx.AKQJTxxx.Ax.8 0.08840',
  'AKQxxx.Txxx.x.Qx JT..KQ98xxx.T8xx 9xx.AKQJ9xx.AJx. 0.08791',
  'Jxxx.Qxx.KQJxxx. KQTxx.9.98x.Q8xx A9.AKJTxxx.Ax.Tx 0.08471',
  'QJx.K9xxx.Q8x.Qx KTxxx..AJxx.T8xx A9x.AQJTxx.K9xx. 0.08400',
  'QTxx.Jxx.AJ9x.QT AKxxx.x.8xx.8xxx J9.AKQT9xx.KQxx. 0.08238',
  'QTxxxxx.Ax.QJx.x 9.9x.A98xxx.T8xx AKJ.KQJTxxx.Kx.Q 0.08155',
  'Jxxx.AQxx.QJ8xx. AQTxx.x.9xx.8xxx K9.KJT9xx.AKx.QT 0.08108'],
 'hcp': -1,
 'shape': -1}

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'D3',
 'candidates': [{'card': 'D3',
   'insta_score': 0.66,
   'expected_tricks_dd': 1.71,
   'p_make_contract': 0.13,
   'expected_score_dd': -372},
  {'card': 'D8',
   'insta_score': 0.339,
   'expected_tricks_dd': 1.71,
   'p_make_contract': 0.13,
   'expected_score_dd': -372},
  {'card': 'DQ',
   'insta_score': 0.0,
   'expected_tricks_dd': 1.52,
   'p_make_contract': 0.13,
   'expected_score_dd': -378}],
 'samples': ['8x.J8.Tx.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axx.Axxxxxx.K9.x 0.785',
  '8x.8x.T9.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axx.AJxxxxx.Kx.x 0.642',
  '8xx.J.Tx.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.A8xxxxxx.K9.x 0.641',
  '8xx.8x.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJxxxxx.K9x.x 0.637',
  '8xx.J8.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.Axxxxxx.K9x.x 0.635',
  '8.Jx.T9x.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axxx.A8xxxxx.K.x 0.632',
  '8xx..T9x.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxxxx.K.x 0.630',
  '8xx.8.T9.A

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)

In [12]:
card_by_card = CardByCard(*board, models, sampler, False)

In [13]:
card_by_card.analyze()

analyzing the bidding
1C OK
2H OK
X ? NN-value: PASS 0.873
4H ? NN-value: PASS 0.346
4S ? NN-value: X 0.728
PASS OK
PASS OK
PASS OK
analyzing the play
DA
D3 OK
D5 OK
D6 OK
C8 OK
C3 OK
CJ OK
CA OK
S4 ? losing: 0.22
S5 OK
S8 OK
SJ OK
H5 OK
HJ OK
HQ OK
HA OK
S2 ? losing: 0.25
SA OK
ST OK
S3 OK
H2 OK
SK OK
H4 OK
H3 OK
D8 OK
D2 OK
DJ OK
S6 OK
SQ OK
C6 OK
H7 OK
S7 OK
H8 OK
C7 OK
HK OK
S9 OK
C5 OK
C2 OK
CT OK
HT OK
CK OK
H9 OK
D7 OK
C4 OK
DQ OK
D4 OK
DK OK
H6 OK


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [14]:
card_by_card.cards['S4'].to_dict()

{'card': 'S4',
 'candidates': [{'card': 'HJ',
   'insta_score': 0.325,
   'expected_tricks_dd': 7.81,
   'p_make_contract': 0.0,
   'expected_score_dd': -60},
  {'card': 'D8',
   'insta_score': 0.046,
   'expected_tricks_dd': 7.72,
   'p_make_contract': 0.0,
   'expected_score_dd': -64},
  {'card': 'DQ',
   'insta_score': 0.113,
   'expected_tricks_dd': 7.71,
   'p_make_contract': 0.0,
   'expected_score_dd': -64},
  {'card': 'S4',
   'insta_score': 0.061,
   'expected_tricks_dd': 7.59,
   'p_make_contract': 0.0,
   'expected_score_dd': -70},
  {'card': 'ST',
   'insta_score': 0.0,
   'expected_tricks_dd': 7.55,
   'p_make_contract': 0.0,
   'expected_score_dd': -72},
  {'card': 'CK',
   'insta_score': 0.107,
   'expected_tricks_dd': 7.48,
   'p_make_contract': 0.0,
   'expected_score_dd': -76},
  {'card': 'SK',
   'insta_score': 0.09,
   'expected_tricks_dd': 6.89,
   'p_make_contract': 0.0,
   'expected_score_dd': -106},
  {'card': 'C7',
   'insta_score': 0.222,
   'expected_tricks_d

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [15]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'candidates': [{'card': 'HK',
   'insta_score': 0.841,
   'expected_tricks_sd': 7.85,
   'p_make_contract': 0.11},
  {'card': 'Sx',
   'insta_score': 0.078,
   'expected_tricks_sd': 7.71,
   'p_make_contract': 0.09}],
 'samples': ['K8xxx.8x.KQx.KQx Tx.Axxx.98x.T9xx AQJ9.J.Jxx.AJxxx 0.74208',
  'AJxx.x.K98xx.Jxx 9x.AJ8xx.Qxx.9xx KQT8x.x.J.AKQTxx 0.73814',
  'AK8x..KQJ8xx.Txx JT.J8xxx.9xx.Jxx Q9xxx.Ax..AKQ9xx 0.73182',
  'AJ9xx.J8.KJ98x.x Kx.Axxx.xx.Qxxxx QT8x.x.Qx.AKJT9x 0.71890',
  'KTxx.8x.KQJ9x.Ax 98.AJxx.xxx.QTxx AQJxx.x.8.KJ9xxx 0.71208',
  'AJ9xx.8.Kxxx.KJT Tx.Axxxx.Q9x.xxx KQ8x.J.J8.AQ9xxx 0.68108',
  'Kxxx.A.KQJ8.KQxx T8.J8xxxx.xxx.9x AQJ9x..9x.AJTxxx 0.67808',
  'QJTx.Jx.KJ8x.QJx Axx.Axxx.9xx.Txx K98x.8.Qx.AK9xxx 0.67756',
  'AJ98xx.x.K98xx.K Q.J8xxx.Jx.QJxxx KTxx.A.Qx.AT9xxx 0.67302',
  'QJTxx.J.KQ9.J9xx 9x.A8xxxx.8xx.AQ AK8x..Jxx.KTxxxx 0.67148',
  'Qxxxx..K8xxx.KQJ 98.J8xxx.Q9x.9xx AKJT.Ax.J.ATxxxx 0.66292',
  'KQJ98.x.KQJ8.Txx xx.J8xxx.9x.KJxx ATxx.A.xxx.AQ9